In [2]:
import pandas as pd
import numpy as np
import copy
from pandas.api.types import is_string_dtype

In [3]:
# Receive datafile
rawdata_file = r"/Users/kathleenszabo/Documents/Cornell Tech/Product Studio/Practice_data.xlsx"
# rawdata_file = r"/Users/kathleenszabo/Documents/Cornell Tech/Product Studio/b2bDataAnalysisDataset.xlsx"
rawdata = pd.read_excel(rawdata_file)
categorical_or_ordered = ['Ordered', 'Categorical', 'Ordered', 'Categorical', 'Categorical']

In [4]:
def get_num_features(data):
    return data.shape[1]

In [5]:
def get_num_observations(data):
    return data.shape[0]

In [6]:
def get_feature_names(data):
    return data.columns

In [7]:
def fill_missing_data(data, categorical_or_ordered):
    
    filled_data = data.copy()
    num_columns = get_num_features(filled_data)
    
    for i in range(num_columns):
        
        filler = 0
        
        # If it's a categorical variable, identify the mode
        if categorical_or_ordered[i] == "Categorical":
            filler = filled_data.iloc[:,i].mode()[0]
        
        elif categorical_or_ordered[i] == "Not applicable":
            filler = "Not applicable"
        
        # If it's a numeric variable, identify the mean
        else:
            filler = filled_data.iloc[:,i].mean()
        
        # Fill blanks in that column with the appropriate value
        filled_data.iloc[:,i] = filled_data.iloc[:,i].fillna(filler)
        
    
    return filled_data

filled_data = fill_missing_data(rawdata, categorical_or_ordered)

In [8]:
def convert_categorical_to_dummies(data, categorical_or_ordered):
    
    final_data = data.copy()
    final_data_cols = get_feature_names(final_data)
    dummy_columns = []
    columns_to_drop = []
    
    for i in range(len(categorical_or_ordered)):
        if categorical_or_ordered[i] == "Categorical":
            dummies = pd.get_dummies(final_data.iloc[:,i])
                        
            new_column_names = []
            for j in range(len(dummies.columns)):
                new_column_names.append(str(str(final_data_cols[i]) + "_" + str(dummies.columns[j])))
            
            if final_data_cols[i] == "Salesperson Name":
                new_column_names = []
                for j in range(len(dummies.columns)):
                    new_column_names.append(str(str("SP_") + str(dummies.columns[j])))
            
            dummies.columns = new_column_names
            dummy_columns.append(dummies)
            columns_to_drop.append(final_data_cols[i])
    
    for i in range(len(dummy_columns)):
        final_data = pd.concat([final_data, dummy_columns[i]], axis=1)
    final_data = final_data.drop(columns_to_drop, axis=1)

    return final_data

In [9]:
def process_data(rawdata, categorical_or_ordered):
    
    filled_data = fill_missing_data(rawdata, categorical_or_ordered)
    final_data = convert_categorical_to_dummies(filled_data, categorical_or_ordered)

    return final_data

final_data = process_data(rawdata, categorical_or_ordered)